In [108]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
from blitz.modules import BayesianLinear
from IPython import display
from torchvision import datasets, transforms
from res.plot_lib import plot_data, plot_model, set_default
from matplotlib import pyplot as plt
from scipy import interpolate

In [121]:
set_default()

In [122]:
presgrid=pd.read_csv("../dataset/CY_doy_pres_limit.csv", "\t")

In [123]:
lat_pivot=np.linspace(-90.5, 90.5, 182)
lon_pivot=np.linspace(-180.5, 180.5, 362)

In [124]:
presgrid=presgrid.to_numpy()
presgrid=presgrid[:, 1:]

In [125]:
x,y=np.meshgrid(lat_pivot,lon_pivot)

In [126]:
prespivot=interpolate.interp2d(x,y,presgrid)

/home/gpietrop/anaconda3/envs/blitz/lib/python3.6/site-packages/scipy/interpolate/_fitpack_impl.py:977: RuntimeWarning: No more knots can be added because the additional knot would
coincide with an old one. Probable cause: s too small or too large
a weight to an inaccurate data point. (fp>s)
	kx,ky=1,1 nx,ny=65,80 m=65884 fp=17529338.937876 s=0.000000
  warnings.warn(RuntimeWarning(_iermess2[ierm][0] + _mess))


In [127]:
def fsigmoid(pre, lat, lon):
    res=1/(1+np.exp((pre-prespivot(lat,lon))/50))
    return res

In [141]:
data=pd.read_csv("../dataset/data_CT.csv")
mont_dict = {'Jan':1,'Feb':2,'Mar':3,'Apr':4,'May':5,'Jun':6,'Jul':7,'Aug':8,'Sep':9,'Oct':10,'Nov':11,'Dec':12}

dataset=data[data.categ=='training']          
validation=data[data.categ=='validation']

out_d=dataset['tcarbn'].to_numpy()
in_d=dataset[['date','date', 'date','latitude', 'longitude', 'pres', 'temp', 'doxy', 'psal']].to_numpy()

out_v=validation['tcarbn'].to_numpy()
in_v=validation[['date','date', 'date', 'latitude', 'longitude', 'pres', 'temp', 'doxy', 'psal']].to_numpy()

In [142]:
def fixd(dataset):
    for i in range(len(dataset[:,0])):
        dataset[i,0] = int(dataset[i,0][7:11])
        dataset[i,1] = mont_dict[dataset[i,1][3:6]]
        dataset[i,2] = int(dataset[i,2][0:2])

fixd(in_d)
fixd(in_v)
in_d=in_d.astype('float64')
in_v=in_v.astype('float64')

In [143]:
data = torch.from_numpy(in_d)
target = torch.from_numpy(out_d)
data=data.float()
target=target.float()

In [145]:
def prep_dataset(data):
    fsigmoi=[]
    for i in range(len(data[:,0])):
        if data[i,4]>180:
            data[i,4]=data[i,4]-360                                                  #longitude
        fsigmoi.append(fsigmoid(data[i,5], data[1,3], data[i,4]))
    data[:,3]=data[:,3]/90                                                           #latitude [-90,90]/90
    data[:,5]=(data[:,5]/20000) + (1 / ((1+np.exp(-(data[:,5])/300))**3))            #pressure
    for i in range(len(data[:,0])):
        day=data[i,2].item()
        month=data[i,1]*30
        month=month.item()
        time=day+month
        data[i,1]=np.cos(time*2*np.pi/365)*fsigmoi[i]
        data[i,2]=np.sin(time*2*np.pi/365)*fsigmoi[i]
    return data

data=prep_dataset(data)

In [7]:
def norm_fun_d(data):
    mean=[ data[:,i].mean() for i in range(data.size()[1]) ]
    std =[ data[:,i].std() for i in range(data.size()[1]) ]
    for i in range(data.size()[1]):
        data[:,i]=2/3*(data[:,i]-mean[i])/std[i] 
    return data, mean, std

def norm_fun_t(target):
    mean=target.mean()
    std=target.std()
    #target=2/3*(target-mean)/std
    return mean, std #target, mean, std

data, mean_data, std_data=norm_fun_d(data)
#mean_ct, std_ct =norm_fun_t(target)
#target

In [8]:
b1=open("../dataset/poids_ct/poids_CT_b1_1.txt")
b2=open("../dataset/poids_ct/poids_CT_b2_1.txt")
b3=open("../dataset/poids_ct/poids_CT_b3_1.txt")
IW=open("../dataset/poids_ct/poids_CT_IW_1.txt")
IW=IW.read()
LW1=open("../dataset/poids_ct/poids_CT_LW1_1.txt")
LW1=LW1.read()
LW2=open("../dataset/poids_ct/poids_CT_LW2_1.txt")
LW2=LW2.read()

b1=np.array([float(i) for i in b1.read().splitlines()])
b2=np.array([float(i) for i in b2.read().splitlines()])
b3=np.array([float(i) for i in b3.read().splitlines()])
IW=np.array([item.split() for item in IW.split('\n')[:-1]])
LW1=np.array([item.split() for item in LW1.split('\n')[:-1]])
LW2=np.array([item.split() for item in LW2.split('\n')[:-1]])
IW=IW.astype(np.float)
LW1=LW1.astype(np.float)
LW2=LW2.astype(np.float)

b1=torch.from_numpy(b1).double()
b2=torch.from_numpy(b2).double()
b3=torch.from_numpy(b3).double()
IW=torch.from_numpy(IW).double()
LW1=torch.from_numpy(LW1).double()
LW2=torch.from_numpy(LW2).double()

In [9]:
def custom_MF(x):
    A=4/3
    a=1.7159
    res=A*(np.exp(a*x)-1)/(np.exp(a*x)+1)
    return res

In [10]:
sam=torch.ones(9,1)
for i in range(9):
    sam[i,0]=data[0,i]
    sam=sam.double()

In [11]:
pr=torch.matmul(IW,sam)
pr=pr.transpose(0,1)
su=b1+pr
res=custom_MF(su)
res=res.transpose(0,1)

pr2=torch.matmul(LW1, res)
pr2=pr2.transpose(0,1)
su2=pr2+b2
res2=custom_MF(su2)
res2=res2.transpose(0,1)

pr3=torch.matmul(LW2, res2)
pr3=pr3.transpose(0,1)
su3=pr3+b3

In [12]:
mean=torch.tensor([2301.0800])
std=torch.tensor([42.060267])

In [13]:
ct_output=1.5*su3*std + mean

In [14]:
ct_output

tensor([[2316.2331]], dtype=torch.float64)